In [1]:
from pydna.dseqrecord import Dseqrecord
dsr = Dseqrecord("ccccGGATCCatgccctaaGGATCCaaaa")
dsr.add_feature(10,19) # a small gene: atgccctaa
dsr.figure()

Dseqrecord(-29)
ccccGGATCCatgccctaaGGATCCaaaa
ggggCCTAGGtacgggattCCTAGGtttt

In [2]:
from Bio.Restriction import BamHI # cuts GGATCC
a,b,c  = dsr.cut(BamHI)

print(a.figure())
print()
print(b.figure())
print()
print(c.figure())

Dseqrecord(-9)
ccccG    
ggggCCTAG

Dseqrecord(-19)
GATCCatgccctaaG    
    GtacgggattCCTAG

Dseqrecord(-9)
GATCCaaaa
    Gtttt


In [3]:
vector = Dseqrecord("aatgtttttcgctgacaatcataatAGATCTtgctatgcatcatcgatct", 
                    circular=True)

from Bio.Restriction import BglII # cuts AGATCT

linear_vector = vector.linearize(BglII)

rec_vector = (linear_vector + b).looped().synced(vector)

rec_vector.figure()

Dseqrecord(o65)
aatgtttttcgctgacaatcataatAGATCCatgccctaaGGATCTtgctatgcatcatcgatct
ttacaaaaagcgactgttagtattaTCTAGGtacgggattCCTAGAacgatacgtagtagctaga